In [7]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome import service as fs
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

# チェックイン日付等をconfig.pyから読み込み
import config

# 日付系変数の設定
# 定数からチェックイン年月日を取得
checkin_date_array = config.checkin_date.split('/')
checkin_year = checkin_date_array[0]
checkin_month = checkin_date_array[1]
# 1〜９月だったら一桁にする
checkin_month_sd = checkin_month.lstrip("0")
checkin_day = checkin_date_array[2]
# 1〜９日だったら一桁にする
checkin_day_sd = checkin_day.lstrip("0")

# 定数からチェックアウト年月日を取得
checkout_date_array = config.checkout_date.split('/')
checkout_year = checkout_date_array[0]
checkout_month = checkout_date_array[1]
# 1〜９月だったら一桁にする
checkout_month_sd = checkout_month.lstrip("0")
checkout_day = checkout_date_array[2]
# 1〜９日だったら一桁にする
checkout_day_sd = checkout_day.lstrip("0")

# 泊数を計算
from datetime import datetime
#int型に変換しないとdatetimeが機能しない
dt1 = datetime( int(checkin_year), int(checkin_month_sd), int(checkin_day_sd) )
dt2 = datetime( int(checkout_year), int(checkout_month_sd), int(checkout_day_sd) )
dt3 = dt2- dt1
stay_period = dt3.days

In [8]:
# Access Denied対策（アパホテル）
from selenium.webdriver.chrome.options import Options
options = Options()
# options.add_argument("start-maximized") #chromeを最大化モードで開く。
options.add_argument('--disable-gpu') #省メモリ化ここから
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--remote-debugging-port=9222') #省メモリ化ここまで
options.add_argument('--disable-blink-features=AutomationControlled')#自動化を無効化する。
options.add_experimental_option("excludeSwitches", ["enable-automation"]) # 「Chromeは自動テストソフトウェアによって制御されています」を消す。
options.add_experimental_option('useAutomationExtension', False) # 自動拡張を無効化する。
options.add_argument('Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36') 

# ドライバー指定でChromeブラウザを開く 
chrome_service = fs.Service(executable_path=config.CHROMEDRIVER) 
driver = webdriver.Chrome(service=chrome_service, options=options)

In [9]:
# 1件目のホテルへアクセス
driver.get(config.target_url_1)

# データ表示まで待つ
# sleep(2) #2件目テストのため短縮

# チェックイン欄をクリックしてカレンダーを出す
checkin_input = driver.find_element(by=By.NAME, value="checkin")
checkin_input.click()


#　カレンダーのチェックイン日付をクリックするためにクラス名を指定。
for_in_XPATH_ym = 'ym_' + checkin_year + '_' + checkin_month

# カレンダーからチェックイン日付の要素を取得して、クリック
checkin_date = driver.find_element(By.XPATH, f"""//div[@id='abi_cal_date']/table[@class='{for_in_XPATH_ym}']/tbody//td[contains(text(), '{checkin_day_sd}')]""")
checkin_date.click()


#　カレンダーのチェックアウト日付をクリックするためにクラス名を指定。
for_out_XPATH_ym = 'ym_' + checkout_year + '_' + checkout_month

# カレンダーからチェックアウト日付の要素を取得して、クリック
checkout_date = driver.find_element(By.XPATH, f"""//div[@id='abi_cal_date']/table[@class='{for_out_XPATH_ym}']/tbody//td[contains(text(), '{checkout_day_sd}')]""")
checkout_date.click()


# 「空室検索」ボタンをクリック
submit_btn = driver.find_element(By.CLASS_NAME, "btn_submit")
submit_btn.click()

# 画面遷移を待つ
sleep(5)

plan_list = driver.find_elements(By.CLASS_NAME, "plan-list")

for plan in plan_list:
    # プランのタイトルに「ショート」が含まれていたらスキップして次のプランを見る。
    if "ショート" in plan.text:
        continue
    # ショートステイでなければ料金を取得
    else:
        # ¥マークで区切り。料金を数値で取得。
        price_array = plan.text.split('￥')
        
        # 改行文字を取り除く。
        price_1 = price_array[1].replace('\n','')
        
        # 料金を取得したらforループ終了
        break
    

In [10]:
# 2件目のホテルへアクセス
driver.get(config.target_url_2)

# データ表示まで待つ
sleep(3)

# チェックイン欄をクリックしてカレンダーを出す
checkin_input = driver.find_element(By.CLASS_NAME, "flatpickr-input")
checkin_input.click()



# カレンダーからチェックイン日付の要素を取得して、クリック
checkin_month = driver.find_element(By.XPATH, f"""//div[@class='flatpickr-current-month']/select[@class='flatpickr-monthDropdown-months']/option[contains(text(), '{checkin_month_sd}月')]""")
checkin_month.click()
#　カレンダーのチェックイン日付をクリックするためにクラス名を指定
for_in_XPATH_label = checkin_month_sd + '月 ' + checkin_day_sd  + ', ' + checkin_year
checkin_date = driver.find_element(By.XPATH, f"""//div[@class='dayContainer']/span[@aria-label='{for_in_XPATH_label}']""")
checkin_date.click()



# カレンダーからチェックアウト日付の要素を取得して、クリック
checkout_month = driver.find_element(By.XPATH, f"""//div[@class='flatpickr-current-month']/select[@class='flatpickr-monthDropdown-months']/option[contains(text(), '{checkout_month_sd}月')]""")
checkout_month.click()
#　カレンダーのチェックアウト日付をクリックするためにクラス名を指定
for_out_XPATH_label = checkout_month_sd + '月 ' + checkout_day_sd  + ', ' + checkout_year
checkout_date = driver.find_element(By.XPATH, f"""//div[@class='dayContainer']/span[@aria-label='{for_out_XPATH_label}']""")
checkout_date.click()

# 「検索」ボタンをクリック
submit_btn = driver.find_element(By.XPATH, "//button[contains(text(), '検索')]")
submit_btn.click()

# 画面遷移を待つ
sleep(5)

# iframe移動
driver.switch_to.frame("tripla-booking-widget-window")

# 予約するボタンの要素を取得できるかテスト
# 要素指定できないのはモーダルウィンドウiframeのせいだった。ハンドル表示のテスト
# print(driver.window_handles)
# ソース内容の確認
# print(driver.page_source)

# 安い順に並び替える
sort_list = driver.find_element(By.XPATH, "//div[@class='booking-wrapper']//div[@data-cy='sort-dropdown']")
sort_list.click()
submit_option = driver.find_element(by=By.LINK_TEXT, value="価格の安い順")
submit_option.click()

# 絞り込みで禁煙を選択
sort_list = driver.find_element(By.XPATH, "//div[@class='booking-wrapper']//div[@data-cy='filter-dropdown']")
sort_list.click()
check_option = driver.find_element(By.XPATH, "//div[@class='filters-container']//label[@for='__BVID__39']")
check_option.click()
submit_option = driver.find_element(By.XPATH, "//div[@class='filters-container']//button[@data-cy='filter-select-btn']")
submit_option.click()

#lazy load対策
while True:
    # 初回アクセス直後のスクリーンの高さを取得
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)

    # 現在のスクリーンの高さを取得
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    

# 価格を変数に取り込む
price_list = driver.find_elements(By.CLASS_NAME, "price")
price_2 = price_list[0].text.lstrip("¥")

In [11]:
# 3件目のホテルへアクセス
driver.get(config.target_url_3)

# データ表示まで待つ
sleep(3)

# 「 Cookieを受け入れる」をクリック
accept_privacy = driver.find_element(By.ID, "onetrust-accept-btn-handler")
accept_privacy.click()

# 「カレンダーから予約」をクリック
menu_calendar = driver.find_element(By.XPATH, "//div[@class='action-list']/ul[@class='hotel_menulist']/li/a[@class='calendar']/span")
menu_calendar.click()

# データ表示まで待つ
sleep(3)

# 泊数が１だったら連泊以外のプランを選択
if stay_period < 2:
    plan_value = '3' #素泊まり一泊プラン
    plan_name_list = driver.find_element(By.ID, 'plan_name')
    select = Select(plan_name_list)
    select.select_by_value('3')
    # 宿泊数１を選択
    stay_count_list = driver.find_element(By.ID, 'stay_count')
    select = Select(stay_count_list)
    select.select_by_value('1')
else:
    plan_value = '57' #素泊まり連泊プラン


# チェックイン日の料金を取得。
#　カレンダーのチェックイン日付をクリックするためにクラス名を指定。末尾の3はたぶん宿泊プランのvalue。
for_in_XPATH_res = 'reserve("' + config.checkin_date.replace('/','-') + '","' + plan_value + '");'

### 2022/04/22現在ここで足踏み。下記find_elementで要素を取得できていない。

# 要素が読み込まれていない？
#lazy load対策
while True:
    # 初回アクセス直後のスクリーンの高さを取得
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(2)

    # 現在のスクリーンの高さを取得
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    

# ソース内容の確認
# print(driver.page_source)

# カレンダーからチェックイン日付の要素を取得して、宿泊費を変数に格納
price_list = driver.find_element(By.XPATH, f"""//a[@onclick='{for_in_XPATH_res}']/div/span[@class='price']""")
price_list_raw = price_list.text.split('円')
price_3 = price_list_raw[0]

# th class名cl-month内のテキストがチェックイン月でなかった場合に次の月へ進む。（後回し）


In [12]:
driver.quit()

In [13]:
print("滞在期間：" + config.checkin_date + " 〜 " + config.checkout_date)
print("費用：")

#価格表示
print(config.hotel_name_1 + ": " + price_1 + "円")
print(config.hotel_name_2 + ": " + price_2 + "円")
print(config.hotel_name_3 + ": " + price_3 + "円")

滞在期間：2022/04/27 〜 2022/04/28
費用：
コンフォートイン六本木: 5,600円
アクトホテル六本木: 6,350円
アパホテル六本木駅前: 7,000円
